In [1]:
import os
import json
import pandas as pd
from ultralytics import YOLO


In [2]:

# --- CONFIGURATION ---
# Chemin vers votre dossier principal
dataset_dir = "Data" 

# Chargement du modèle
# On utilise le modèle nano (léger) standard. 
# Il est entraîné sur COCO, donc on détectera la classe 0 ("person") qui correspond à la main/bras.
model = YOLO('yolov8n.pt') 

# Structures de données pour l'export
# Format JSON (Le plus important pour Edge Impulse CLI)
ei_labels = {
    "version": 1,
    "type": "bounding-box",
    "boundingBoxes": {}
}
# Format CSV (Pour votre demande spécifique)
csv_data = []

print(f"--- Analyse du dossier '{dataset_dir}' ---")

if not os.path.exists(dataset_dir):
    print(f"ERREUR: Le dossier '{dataset_dir}' est introuvable.")
    exit()

# Parcours des dossiers de classes (A.class, B.class, etc.)
for class_folder in sorted(os.listdir(dataset_dir)):
    folder_path = os.path.join(dataset_dir, class_folder)
    
    # Vérifie que c'est bien un dossier
    if not os.path.isdir(folder_path):
        continue
        
    # --- 1. NETTOYAGE DU LABEL ---
    # Transforme "A.class" en "A"
    if class_folder.endswith(".class"):
        label = class_folder.replace(".class", "")
    else:
        label = class_folder # Cas où le dossier serait déjà nommé "A"
    
    print(f"Traitement du dossier : {class_folder} -> Label : {label}")
    
    # Parcours des images (00000.jpg, 00001.jpg...)
    for image_file in os.listdir(folder_path):
        if not image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            continue
            
        full_path = os.path.join(folder_path, image_file)
        
        # --- IMPORTANT : GESTION DES DOUBLONS ---
        # Comme vos fichiers s'appellent tous 00000.jpg dans chaque dossier,
        # nous utilisons le chemin relatif (ex: "A.class/00000.jpg") comme clé unique.
        relative_path = os.path.join(class_folder, image_file).replace("\\", "/") # Force les slashs pour compatibilité
        
        # --- 2. INFERENCE YOLO ---
        # conf=0.4 : On garde les détections avec >40% de confiance
        # classes=[0] : On ne garde que la classe "personne" (votre main)
        results = model.predict(full_path, save=False, conf=0.4, classes=[0], verbose=False)
        
        for result in results:
            # On vérifie si une boîte a été trouvée
            if len(result.boxes) > 0:
                # On prend la boîte avec le meilleur score (la première)
                box = result.boxes[0]
                
                # Coordonnées YOLO (x_min, y_min, x_max, y_max)
                x_min, y_min, x_max, y_max = box.xyxy[0].tolist()
                
                # Conversion pour Edge Impulse (x, y, largeur, hauteur)
                x = int(x_min)
                y = int(y_min)
                w = int(x_max - x_min)
                h = int(y_max - y_min)
                
                # Ajout au JSON (Clé = chemin relatif)
                if relative_path not in ei_labels["boundingBoxes"]:
                    ei_labels["boundingBoxes"][relative_path] = []
                
                ei_labels["boundingBoxes"][relative_path].append({
                    "label": label,
                    "x": x,
                    "y": y,
                    "width": w,
                    "height": h
                })
                
                # Ajout au CSV
                csv_data.append([image_file, label, x, y, w, h, relative_path])

# --- 3. SAUVEGARDE DES FICHIERS ---

# Export JSON (Fichier clé pour l'import auto)
json_path = os.path.join(dataset_dir, "bounding_boxes.labels")
with open(json_path, 'w') as f:
    json.dump(ei_labels, f, indent=4)

# Export CSV
df = pd.DataFrame(csv_data, columns=['filename', 'label', 'x', 'y', 'width', 'height', 'path'])
csv_path = "bounding_boxes.csv"
df.to_csv(csv_path, index=False)

print("\n--- TERMINE ---")
print(f"1. Fichier JSON généré dans : {json_path}")
print(f"   (C'est ce fichier que Edge Impulse utilisera pour placer les boîtes automatiquement)")
print(f"2. Fichier CSV généré : {csv_path} (Pour votre vérification)")

--- Analyse du dossier 'Data' ---
Traitement du dossier : A.class -> Label : A
Traitement du dossier : B.class -> Label : B
Traitement du dossier : C.class -> Label : C
Traitement du dossier : D.class -> Label : D
Traitement du dossier : E.class -> Label : E
Traitement du dossier : F.class -> Label : F
Traitement du dossier : G.class -> Label : G
Traitement du dossier : H.class -> Label : H
Traitement du dossier : I.class -> Label : I
Traitement du dossier : K.class -> Label : K
Traitement du dossier : L.class -> Label : L
Traitement du dossier : M.class -> Label : M
Traitement du dossier : N.class -> Label : N
Traitement du dossier : O.class -> Label : O
Traitement du dossier : P.class -> Label : P
Traitement du dossier : Q.class -> Label : Q
Traitement du dossier : R.class -> Label : R
Traitement du dossier : S.class -> Label : S
Traitement du dossier : T.class -> Label : T
Traitement du dossier : U.class -> Label : U
Traitement du dossier : V.class -> Label : V
Traitement du dossier

In [3]:
print("Finis")

Finis
